# Setup

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# check environment
import os
print(f'Conda Environment: ' + os.environ['CONDA_DEFAULT_ENV'])

from platform import python_version
print(f'python version: {python_version()}')

Conda Environment: ROI_env
python version: 3.8.13


In [3]:
%load_ext autoreload
%autoreload 2

import importlib.util
import glob
import pathlib
from pathlib import Path
from umap import UMAP
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ShuffleSplit, StratifiedShuffleSplit
import scipy.stats
import scipy.signal
from kymatio import Scattering2D
import json
import torchvision
import torch
from tqdm.notebook import tqdm, trange
import sys
import pickle
import pandas as pd
import shutil
import h5py
# import figgen as fg
import sys
import time
import gc
import numpy as np

print('Numpy Version:', np.__version__)
# print('TorchVision Version:',torchvision.__version__)

dir_github = Path(r'/n/data1/hms/neurobio/sabatini/josh/github_repos/').resolve()

import sys
sys.path.append(str(dir_github))

from ROICaT.roicat import helpers, ROInet
# from ROICaT.roicat.tracking import data_importing, visualization, alignment, blurring, ROInet, scatteringWaveletTransformer, similarity_graph, clustering
from ROICaT.roicat.classification import data, preprocess, classify, evaluate, visualize
import multiprocessing as mp

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Numpy Version: 1.21.6


In [4]:
tic = time.time()
toc = {}
toc['start'] = time.time() - tic

# Import Paths

In [5]:
def spr(*directory_list):
    for dir_num, directory in enumerate(directory_list):
        if dir_num == 0:
            full_directory = Path(directory)
        else:
            full_directory = full_directory / directory
    return str(full_directory.resolve())

In [6]:
def print_list(l):
    for item in l:
        print(item)

In [7]:
# dir_allOuterFolders = Path(r"/media/rich/bigSSD/other lab data/Harnett_lab/ROI_Tracking/Vincent_Valerio/4th_email/AllStatFiles/rbp16").resolve()
# # dir_allOuterFolders = Path(r"/media/rich/bigSSD/res2p/scanimage data/round 5 experiments/mouse 2_6/just_stat_files").resolve()

# folders_allSessions = natsort.natsorted(helpers.get_dir_contents(dir_allOuterFolders)[0])

# dir_allS2pFolders = [dir_allOuterFolders / folder for folder in folders_allSessions]

# pathSuffixToStat = 'plane0/stat.npy'
# pathSuffixToOps = 'plane0/ops.npy'
# # pathSuffixToStat = 'stat.npy'
# # pathSuffixToOps = 'ops.npy'

# paths_allStat = np.array([path / pathSuffixToStat for path in dir_allS2pFolders])[:]
# paths_allOps  = np.array([path / pathSuffixToOps for path in dir_allS2pFolders])[:]
# # paths_allStat = np.array([path / pathSuffixToStat for path in dir_allS2pFolders])
# # paths_allOps  = np.array([path / pathSuffixToOps for path in dir_allS2pFolders])

# print(f'folder names of all sessions: \n{folders_allSessions}')
# print(f'paths to all stat files: \n{paths_allStat}')


# ---------------------------------------
# || Specify Raw Data Filename Sources ||
# ---------------------------------------
stat_files = spr('/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/data/training/mouse2_6__20210409/stat.npy')
label_files = spr('/n/data1/hms/neurobio/sabatini/josh/github_repos/GCaMP_ROI_classifier/data/training/mouse2_6__20210409/labels_round2_sesh2.npy')

# Import Data

In [8]:
dat = data.Data(
    paths_statFiles=stat_files,
    paths_labelFiles=label_files,
    um_per_pixel=1.0,
    verbose=True,
)

dat.import_statFiles();
dat.import_labelFiles();

# data.import_ROI_spatialFootprints(workers=-1);

Starting: Importing spatial footprints from stat files
Completed: Imported 4898 stat files into class as self.statFiles. Total number of ROIs: 4898. Number of ROI from each file: [4898]
Starting: Importing labels footprints from npy files
Completed: Imported 4898 labels into class as self.labelFiles. Total number of ROIs: 4898. Number of ROI from each file: [4898]


In [9]:
# %matplotlib notebook

# # From ROICaT
# visualization.display_toggle_image_stack(np.concatenate(data.ROI_images, axis=0))

In [10]:
toc['import_data'] = time.time() - tic

# Concatenate / Adjust / Clean Data + Drop Non-Nans

In [11]:
dat.drop_nan_rois();

# Neural Network Embeddings

In [12]:
hash_dict_true = {
    'params': ('params.json', '68cf1bd47130f9b6d4f9913f86f0ccaa'),
    'model': ('model.py', '61c85529b7aa33e0dfadb31ee253a7e1'),
    'state_dict': ('ConvNext_tiny__1_0_best__simCLR.pth', '3287e001ff28d07ada2ae70aa7d0a4da'),
}

roinet = ROInet.ROInet_embedder(
    device='cuda:0',
#     dir_networkFiles='/home/rich/Downloads/ROInet',
    dir_networkFiles='/n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT',
    
#     download_from_gDrive='force_download',
#     download_from_gDrive='force_local',
    download_from_gDrive='check_local_first',
    gDriveID='1D2Qa-YUNX176Q-wgboGflW0K6un7KYeN',
    hash_dict_networkFiles=hash_dict_true,
#     hash_dict_networkFiles=None,
    forward_pass_version='latent',
    verbose=True,
)

roinet.generate_dataloader(
    ROI_images=[dat.statFiles],
    um_per_pixel=dat.um_per_pixel,
    pref_plot=False,
    batchSize_dataloader=8,
    pinMemory_dataloader=True,
    numWorkers_dataloader=mp.cpu_count(),
    persistentWorkers_dataloader=True,
    prefetchFactor_dataloader=2,    
);

roinet.generate_latents();

# roinet.latents
# roinet.dataset
# roinet.net
# roinet.params_model

Successful hash comparison. Found matching files: {'params': '/n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT/params.json', 'model': '/n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT/model.py', 'state_dict': '/n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT/ConvNext_tiny__1_0_best__simCLR.pth'}
Imported model from /n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT/model.py
Loaded params_model from /n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT/params.json
Generated network using params_model
Loaded state_dict into network from /n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT/ConvNext_tiny__1_0_best__simCLR.pth
Loaded network onto device cuda:0
Starting: resizing ROIs


100%|██████████| 1/1 [00:00<00:00,  2.21it/s]
/home/joz608/.conda/envs/ROI_env/lib/python3.8/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 48 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Completed: resizing ROIs
Defined image transformations: Sequential(
  (0): ScaleDynamicRange(scaler_bounds=(0, 1))
  (1): Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
  (2): TileChannels(dim=0)
)
Defined dataset
Defined dataloader
starting: running data through network


100%|██████████| 613/613 [00:12<00:00, 48.61it/s]

completed: running data through network


In [13]:
latents = (roinet.latents)

In [15]:
# np.load('/n/data1/hms/neurobio/sabatini/josh/analysis/data_backup_save/latents.npy')

In [16]:
gc.collect()
torch.cuda.empty_cache()
gc.collect()
torch.cuda.empty_cache()

In [17]:
toc['NN'] = time.time() - tic

# UMAP Embeddings

In [18]:
umap_params = dict(
    n_neighbors=30,
    n_components=2,
    metric='euclidean',
    metric_kwds=None,
    output_metric='euclidean',
    output_metric_kwds=None,
    n_epochs=None,
    learning_rate=1.0,
    init='spectral',
    min_dist=0.1,
    spread=1.0,
    low_memory=True,
    n_jobs=-1,
    set_op_mix_ratio=1.0,
    local_connectivity=1.0,
    repulsion_strength=1.0,
    negative_sample_rate=5,
    transform_queue_size=4.0,
    a=None,
    b=None,
    random_state=None,
    angular_rp_forest=False,
    target_n_neighbors=-1,
    target_metric='categorical',
    target_metric_kwds=None,
    target_weight=0.5,
    transform_seed=42,
    transform_mode='embedding',
    force_approximation_algorithm=False,
    verbose=False,
    tqdm_kwds=None,
    unique=False,
    densmap=False,
    dens_lambda=2.0,
    dens_frac=0.3,
    dens_var_shift=0.1,
    output_dens=False,
    disconnection_distance=None,
    precomputed_knn=(None, None, None),
)

umap = UMAP(**umap_params)
print('Fitting UMAP...')
umap.fit(latents)
print('Generating Embeddings...')
embeddings = umap.transform(latents)
print('Embeddings Generated...')

Fitting UMAP...
Generating Embeddings...
Embeddings Generated...


# Visualize Embeddings

In [19]:
roinet.latents.shape, dat.labelFiles.shape

(torch.Size([4898, 128]), (4898,))

In [20]:
toc['visualize'] = time.time() - tic

# Load / Setup Classifier — ZScore + PCA

In [58]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression

preprocessor = make_pipeline(
    StandardScaler(),
    PCA(),
    LogisticRegression(
        solver='lbfgs',
        fit_intercept=True, 
        class_weight='balanced',
        max_iter=10000,
        C=1
    )
)
preprocessor.fit(latents, dat.labelFiles);

proba = preprocessor.predict_proba(latents);
preds = np.argmax(proba, axis=1);
cm = helpers.confusion_matrix(preds, dat.labelFiles.astype(np.int32), counts=False);

Pipeline(steps=[('standardscaler', StandardScaler()), ('pca', PCA()),
                ('logisticregression',
                 LogisticRegression(C=1, class_weight='balanced',
                                    max_iter=10000))])

In [59]:
cm

In [21]:
preprocessor = preprocess.Preprocessor()
pp = preprocessor.fit_transform_preprocess(latents)

TypeError: __init__() missing 1 required positional argument: 'pipeline'

In [ ]:
preprocessor.transform_preprocess(latents)

In [ ]:
classifier = classify.Classifier(preprocessor)
classifier.fit_classifier(latents, dat.labelFiles);
proba, preds = classifier.classify(latents)

In [ ]:
evaluator = evaluate.Evaluation(classifier)
cm = evaluator.confusion_matrix(latents, dat.labelFiles)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,5))
sns.heatmap(cm, annot=True, annot_kws={"size": 16}, vmax=1., cmap=plt.get_cmap('gray'), ax=ax)
fig.suptitle('Train')

## Train/Holdout Split

In [23]:
# # Train/Holdout Split
# features_train, features_holdout, labels_train, labels_holdout = sklearn.model_selection.train_test_split(features_norm, labels_clean[:features_norm.shape[0]], test_size=0.3)



## Shuffle Split

In [24]:
# # Shuffle Split
# splitter = ShuffleSplit(n_splits=classifier_n_splits)
# all_split_inx = list(splitter.split(features_train))

## Train'/Val Extract

In [25]:
# # # Train'/Val Extract
# trainp_X = [features_train[_[0]] for _ in all_split_inx]
# val_X = [features_train[_[1]] for _ in all_split_inx]
# trainp_y = [labels_train[_[0]] for _ in all_split_inx]
# val_y = [labels_train[_[1]] for _ in all_split_inx]

## Fit Classifier

In [26]:
# # Sweep through regularizations for classifier
# for ic, c in enumerate(C_toUse):
#     cm_trp_lst = []
#     cm_val_lst = []

#     cm_trp_cnt_lst = []
#     cm_val_cnt_lst = []

#     acc_trp_lst = []
#     acc_val_lst = []

#     # Sweep Through Shuffle Splits
#     # trp = Train' or train within single Shuffle Split fold
#     # val = Validation or test set within single Shuffle Split fold
#     for inx_split in trange(len(trainp_X)):
#         tmp_trainp_X = trainp_X[inx_split]
#         tmp_trainp_y = trainp_y[inx_split]

#         tmp_val_X = val_X[inx_split]
#         tmp_val_y = val_y[inx_split]

#         logreg = fg.fit_classifier_logreg(tmp_trainp_X, tmp_trainp_y, max_iter=logistic_max_iter, C=c)

#         cm_trp = fg.cm_classifier_logreg(logreg, tmp_trainp_X, tmp_trainp_y, github_loc=fig_gen_params['github_loc'])
#         cm_val = fg.cm_classifier_logreg(logreg, tmp_val_X, tmp_val_y, github_loc=fig_gen_params['github_loc'])

#         cm_trp_cnt = fg.cm_classifier_logreg(logreg, tmp_trainp_X, tmp_trainp_y, counts=True, github_loc=fig_gen_params['github_loc'])
#         cm_val_cnt = fg.cm_classifier_logreg(logreg, tmp_val_X, tmp_val_y, counts=True, github_loc=fig_gen_params['github_loc'])

#         acc_trp = fg.score_classifier_logreg(logreg, tmp_trainp_X, tmp_trainp_y)
#         acc_val = fg.score_classifier_logreg(logreg, tmp_val_X, tmp_val_y)

#         cm_trp_lst.append(cm_trp)
#         cm_val_lst.append(cm_val)

#         cm_trp_cnt_lst.append(cm_trp_cnt)
#         cm_val_cnt_lst.append(cm_val_cnt)

#         acc_trp_lst.append(acc_trp)
#         acc_val_lst.append(acc_val)
    
    
#     cm_trp_mn = np.mean(cm_trp_lst,axis=0)
#     cm_val_mn = np.mean(cm_val_lst,axis=0)

#     cm_trp_cnt_sm = np.sum(cm_trp_cnt_lst,axis=0)
#     cm_val_cnt_sm = np.sum(cm_val_cnt_lst,axis=0)

#     # Refitting model to all of training / CV data and evaluating on heldout data
#     logreg_refit = fg.fit_classifier_logreg(features_train, labels_train, max_iter=logistic_max_iter, C=c)

#     cm_tr = fg.cm_classifier_logreg(logreg_refit, features_train, labels_train, counts=False, github_loc=fig_gen_params['github_loc'])
#     cm_ho = fg.cm_classifier_logreg(logreg_refit, features_holdout, labels_holdout, counts=False, github_loc=fig_gen_params['github_loc'])

#     cm_tr_cnt = fg.cm_classifier_logreg(logreg_refit, features_train, labels_train, counts=True, github_loc=fig_gen_params['github_loc'])
#     cm_ho_cnt = fg.cm_classifier_logreg(logreg_refit, features_holdout, labels_holdout, counts=True, github_loc=fig_gen_params['github_loc'])

#     acc_tr = fg.score_classifier_logreg(logreg_refit, features_train, labels_train)
#     acc_ho = fg.score_classifier_logreg(logreg_refit, features_holdout, labels_holdout)
    
#     restricted_n_train_results = {}
#     for n_train in n_train_to_use:
#         print('Saving: n_train',n_train)
#         train_size = n_train/features_train.shape[0] if type(n_train) == type(None) and n_train < features_train else None
#         print('Saving: train_size',train_size)
#         if train_size is not None:
#             # Refitting model to n_train data points from training data / and evaluating on heldout data
#             sss = StratifiedShuffleSplit(n_splits=1, train_size=train_size)
#             train_subset_inx, _ = sss.split(features_train, labels_train)[0]
#             features_train_subset, labels_train_subset = features_train[train_subset_inx], labels_train[train_subset_inx]
            
#             logreg_refit = fg.fit_classifier_logreg(features_train_subset, labels_train_subset, max_iter=logistic_max_iter, C=c)
            
#             restricted_n_train_results[f'acc_tr_subset_{n_train}'] = fg.score_classifier_logreg(logreg_refit, features_train_subset, labels_train_subset)
#             restricted_n_train_results[f'acc_tr_{n_train}'] = fg.score_classifier_logreg(logreg_refit, features_train, labels_train)
#             restricted_n_train_results[f'acc_ho_{n_train}'] = fg.score_classifier_logreg(logreg_refit, features_holdout, labels_holdout)
    
#     with h5py.File(fig_gen_params['h5_out_name'], 'a') as f:
#         g = f.create_group(f'creg_{c}')
#         g.attrs['acc_tr'] = np.mean(acc_tr)
#         g.attrs['acc_trp'] = np.mean(acc_trp_lst)
#         g.attrs['acc_val'] = np.mean(acc_val_lst)
#         g.attrs['acc_ho'] = np.mean(acc_ho)
        
#         for n_train in n_train_to_use:
#             print('Saving: n_train',n_train)
#             train_size = n_train/features_train.shape[0] if type(n_train) == type(None) and n_train < features_train else None
#             print('Saving: train_size',train_size)
#             if train_size is not None:
#                 g.attrs[f'acc_tr_subset_{n_train}'] = restricted_n_train_results[f'acc_tr_subset_{n_train}']
#                 g.attrs[f'acc_tr_{n_train}'] = restricted_n_train_results[f'acc_tr_{n_train}']
#                 g.attrs[f'acc_ho_{n_train}'] = restricted_n_train_results[f'acc_ho_{n_train}']

#         gg = g.create_group(f'cm_prc')

#         gg.create_dataset(f'tr', data=cm_tr)
#         gg.create_dataset(f'trp', data=cm_trp_mn)
#         gg.create_dataset(f'val', data=cm_val_mn)
#         gg.create_dataset(f'ho', data=cm_ho)

#         gg = g.create_group(f'cm_count')

#         gg.create_dataset(f'tr', data=cm_tr_cnt)
#         gg.create_dataset(f'trp', data=cm_trp_cnt_sm)
#         gg.create_dataset(f'val', data=cm_val_cnt_sm)
#         gg.create_dataset(f'ho', data=cm_ho_cnt)

#         gg = g.create_group(f'logreg_model')

#         gg.create_dataset(f'coef', data=logreg_refit.coef_)
#         gg.create_dataset(f'int', data=logreg_refit.intercept_)
    
    

## Save Results

In [ ]:
# dir_save = Path('/home/rich/Desktop/').resolve()
dir_save = Path('/n/data1/hms/neurobio/sabatini/josh/analysis/roinet-paper/ROICaT/output/').resolve()
name_save = dir_allOuterFolders.name
path_save = dir_save / (name_save + '.ROICaT.results' + '.pkl')

ROIs = {
    "ROIs_aligned": aligner.ROIs_aligned,
    "ROIs_raw": dat.spatialFootprints,
    "frame_height": dat.FOV_height,
    "frame_width": dat.FOV_width,
    "idx_roi_session": np.where(dat.sessionID_concat)[1]
}

results = {
    "UCIDs": labels,
    "UCIDs_bySession": labels_bySession,
    "ROIs": ROIs,
}

helpers.simple_save(
    obj=results,
    filename=path_save,
    mkdir=True,
)

In [ ]:
toc['saving'] = time.time() - tic

In [ ]:
toc

In [ ]:
# Sweep through regularizations for classifier

    # Sweep Through Shuffle Splits

    # Refitting model to all of training / CV data and evaluating on heldout data

            # Refitting model to n_train data points from training data / and evaluating on heldout data
